---
**License**

 VexpBounce

 Tue Nov 15 12:13:14 2022\
 Copyright  2022\
 Sandro Dias Pinto Vitenti <vitenti@uel.br>

---
---

 MagDustBounce\
 Copyright (C) 2022 Sandro Dias Pinto Vitenti <vitenti@uel.br>

 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

In [ ]:
try:
  import gi
  gi.require_version('NumCosmo', '1.0')
  gi.require_version('NumCosmoMath', '1.0')
except:
  pass

import sys
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm



In [ ]:
Vexp = Nc.HICosmoVexp.new ()
Vexp.props.alphab   = 8.3163e-2
Vexp.props.sigmaphi = 9.0
Vexp.props.dphi     = -9.0e-4
Vexp.props.xb       = 2.0e36
Vexp.props.OmegaL   = 1.0
Vexp.props.Omegac   = 1.0
Vexp.props.H0       = 67.8

tc = Vexp.tau_xe (1.0e15)
reltol = 1.0e-14


In [ ]:
tau_min = Vexp.tau_min ()
tau_max = Vexp.tau_max ()
tau_qt_c = Vexp.tau_qt_c()
tau_qt_e = Vexp.tau_qt_e()
npoints = 5000

tau_a = np.linspace (tau_min, tau_max, npoints)
tau_q_a = np.linspace (tau_qt_c, tau_qt_e, npoints)

print (f"Vexp model prepared in the interval ({tau_min},{tau_max})")
print (f"Vexp quantum dominated interval ({tau_qt_c},{tau_qt_e})")

In [ ]:
plt.figure (figsize=(14, 7))

alpha_a = [Vexp.alpha (tau) for tau in tau_a]

plt.plot (tau_a, alpha_a, label=r"$\alpha$")

plt.grid (visible=True, which='both', linestyle=':', color='0.75', linewidth=0.5)
leg = plt.legend (loc="best", ncol = 2, fontsize=13)

plt.xlabel (r"$\tau$")
plt.show()
pass

In [ ]:
plt.figure (figsize=(14, 7))

phi_a = [Vexp.phi (tau) for tau in tau_a]

plt.plot (tau_a, phi_a, label=r"$\phi$")

plt.grid (visible=True, which='both', linestyle=':', color='0.75', linewidth=0.5)
leg = plt.legend (loc="best", ncol = 2, fontsize=13)

plt.xlabel (r"$\tau$")
plt.show()

In [ ]:
plt.figure (figsize=(14, 7))

RS_a = [Vexp.Ricci_scale (tau) for tau in tau_a]

plt.plot (tau_a, RS_a, label=r"$R_s$")

plt.grid (visible=True, which='both', linestyle=':', color='0.75', linewidth=0.5)
leg = plt.legend (loc="best", ncol = 2, fontsize=13)

plt.xlabel (r"$\tau$")
plt.yscale('log')

plt.show()

In [ ]:
plt.figure (figsize=(14, 7))

x_y_a = np.array ([Vexp.x_y (tau) for tau in tau_a])

plt.plot (tau_a, x_y_a[:,0], label='$x$')
plt.plot (tau_a, x_y_a[:,1], label='$y$')
plt.plot (tau_a, np.hypot(x_y_a[:,0], x_y_a[:,1]), label='$\sqrt{x^2+y^2}$')

plt.grid (visible=True, which='both', linestyle=':', color='0.75', linewidth=0.5)
leg = plt.legend (loc="best", ncol = 2, fontsize=13)

plt.xlabel (r"$\tau$")
plt.yscale('symlog')

plt.show()

In [ ]:
#plt.figure (figsize=(14, 7))

#x_y_a = np.array ([Vexp.x_y (tau) for tau in tau_a])

#plt.plot (tau_a, 2.0 * x_y_a[:,0]**2-1.0, label='$w = 2x^2-1$')

#plt.grid (visible=True, which='both', linestyle=':', color='0.75', linewidth=0.5)
#leg = plt.legend (loc="best", ncol = 2, fontsize=13)

#plt.xlabel (r"$\tau$")
#plt.yscale('symlog')
#plt.ylim(-1.1, 1.1)

#plt.show()

plt.figure (figsize=(14, 7))

x_y_a = np.array ([Vexp.x_y (tau) for tau in tau_q_a])
phi_q_a = [Vexp.phi (tau) for tau in tau_q_a]
phi_q_a = np.array (phi_q_a)

plt.plot (tau_q_a, phi_q_a, label='$\phi$')
plt.plot (tau_q_a, np.exp (-(phi_q_a / 0.08)**2), label=r'$f(\phi)$')
plt.plot (tau_q_a, 2.0 * x_y_a[:,0]**2-1.0, label='$w = 2x^2-1$')

plt.grid (visible=True, which='both', linestyle=':', color='0.75', linewidth=0.5)
leg = plt.legend (loc="best", ncol = 2, fontsize=13)

plt.xlabel (r"$\tau$")
#plt.yscale('symlog')
plt.xlim(tau_qt_c, tau_qt_e)
plt.ylim(-2.1, 2.1)

plt.show()

In [ ]:
gw = Nc.HIPertGW.new ()

gw.set_ti (tau_min)
gw.set_tf (tau_max)
gw.set_k (1.0)
gw.set_reltol (reltol)


In [ ]:

plt.figure (figsize=(14, 7))

deta_dtau_a = np.array ([gw.eval_nu (Vexp, tau, 1) for tau in tau_a])

plt.plot (tau_a, deta_dtau_a, label=r'$\mathrm{d}\eta/\mathrm{d}\tau$')

plt.grid (visible=True, which='both', linestyle=':', color='0.75', linewidth=0.5)
leg = plt.legend (loc="best", ncol = 2, fontsize=13)

plt.xlabel (r"$\tau$")
plt.yscale('log')

plt.show()


In [ ]:
tau_v = Ncm.Vector.new_array (tau_a)
deta_dtau_v = Ncm.Vector.new_array (deta_dtau_a)
s = Ncm.SplineCubicNotaknot.new ()
s.set(tau_v, deta_dtau_v, True)

def eta_tau(tau):
    if tau > 0.0:
        return s.eval_integ (0.0, tau)
    return -s.eval_integ (tau, 0.0)

In [ ]:
plt.figure (figsize=(14, 7))

eta_a = np.array ([eta_tau (tau) for tau in tau_a])

plt.plot (tau_a, eta_a, label='$\eta$')

plt.grid (visible=True, which='both', linestyle=':', color='0.75', linewidth=0.5)
leg = plt.legend (loc="best", ncol = 2, fontsize=13)

plt.xlabel (r"$\tau$")
plt.yscale('symlog', linthresh=1.0e-40)

plt.show()

plt.figure (figsize=(14, 7))

eta_q_a = np.array ([eta_tau (tau) for tau in tau_q_a])

plt.plot (tau_q_a, eta_q_a, label='$\eta$')

plt.grid (visible=True, which='both', linestyle=':', color='0.75', linewidth=0.5)
leg = plt.legend (loc="best", ncol = 2, fontsize=13)

plt.xlabel (r"$\tau$")
#plt.yscale('symlog', linthresh=1.0e-40)

plt.show()

In [ ]:
plt.figure (figsize=(14, 7))

x_a = np.array ([Vexp.x_tau (tau) for tau in tau_a])

plt.plot (tau_a, x_a, label='$\eta$')

plt.grid (visible=True, which='both', linestyle=':', color='0.75', linewidth=0.5)
leg = plt.legend (loc="best", ncol = 2, fontsize=13)

plt.xlabel (r"$\tau$")
#plt.xscale('symlog')
plt.yscale('symlog')

plt.show()

In [ ]:
plt.figure (figsize=(14, 7))

plt.plot (eta_q_a, phi_q_a, label=r"$\phi$")

plt.grid (visible=True, which='both', linestyle=':', color='0.75', linewidth=0.5)
leg = plt.legend (loc="best", ncol = 2, fontsize=13)

plt.xlabel (r"$\tau$")
plt.xlim(-0.4e-38, 0.4e-38)
plt.ylim(-0.5, 0.5)
plt.show()

In [ ]:
x_y_a = np.array ([Vexp.x_y (tau) for tau in tau_q_a])

plt.plot (tau_q_a, 2.0 * x_y_a[:,0]**2-1.0, label='$w = 2x^2-1$')

plt.grid (visible=True, which='both', linestyle=':', color='0.75', linewidth=0.5)
leg = plt.legend (loc="best", ncol = 2, fontsize=13)

plt.xlabel (r"$\tau$")
#plt.yscale('symlog')
plt.ylim(-1.1, 1.1)

plt.show()

In [ ]:
plt.figure (figsize=(14, 7))
plt.plot (eta_a, phi_a)
plt.xscale("symlog", linthresh=1.0e-38)
#plt.xlim((-1.0e4,1.0e4))

In [ ]:
plt.figure (figsize=(14, 7))
plt.plot (eta_q_a, phi_q_a)
#plt.xscale("symlog", linthresh=1.0e-38)
plt.xlim((None, 0.5e-38))